<a href="https://colab.research.google.com/github/ShawnRonin/Coursera_Capstone/blob/master/Capstone5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# London Gym Search
*IBM Data Science Professional Certificate Capstone Project*



**Introduction**

On behalf of stakeholders in a prospective venture to open and gym in the city of London, Ontario, Canada we are undertaking the service of performing a survey of gyms in the city. The stakeholders are interested in finding an area not currently served by a gym to minimize their competition. Opening in a location with a surplus of gyms would require a more significant investment in start-up marketing and require reducing costs to be able to undercut nearby competition. The stakeholders wish to avoid this.


**Business Problem**

Using Foursquare data for the city of London, Ontario, Canada, where could a prospective gym owner find an area not currently being served by an existing fitness business. Fitness is a booming industry and prospective real estate will be moving fast. Does London have the space for another gym. If so, where? 


**Data & Methodology**

Data will be pulled from the Foursquare API, specifically querying for fitness related businesses within a fifteen-kilometer radius of the city centre of London, Ontario, Canada. Searched data will be stored and filtered for applicability and then location data will be plotted on a map to visualize the current state of the fitness industry in London.


In [0]:
import pandas as pd

In [0]:
CLIENT_ID = 'VACLNPU0GQJWRLQFFS53YYJVKOFU104PVLCERJOFTFGELELM' 
CLIENT_SECRET = 'VXZSKZE2GXYP4RTZIGE0400A4Y350PAA2RP0P5OQTLGU3QZS'
VERSION = '20190623'

We will begin by writing a function to query the Foursquare API. The query will be center on the city of interest out to a radius of 15km. We will be specially querying Venues that fit the Fitness category, ID code: 4bf58dd8d48988d175941735

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=15000):
    
    venues_list=[]
    name, lat, lng  = names, latitudes, longitudes
    print(name)
            
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId=4bf58dd8d48988d175941735'.format(
         CLIENT_ID, 
         CLIENT_SECRET, 
         VERSION, 
         lat, 
         lng, 
         15000, 
         100
         )
            
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
       
    # return only relevant information for each nearby venue
    venues_list.append([(
        name, 
        lat, 
        lng, 
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Using the geolocator function, we find the city center of London, Ontario, Canada. We have to be careful not to get it confused with London, England.

In [212]:
address = 'London, Ontario, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London, Ontario, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London, Ontario, Canada are 42.9537654, -81.2291529.


Now we call foursquare to get ftiness venues within a 15km radius of the London, Ontario, Canada city center.

In [213]:
london_venues = getNearbyVenues(names=address,
                                   latitudes=latitude,
                                   longitudes=longitude
                                  )

London, Ontario, Canada


In [214]:
london_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"London, Ontario, Canada",42.953765,-81.229153,YMCA,42.985099,-81.241463,Gym / Fitness Center
1,"London, Ontario, Canada",42.953765,-81.229153,The Fitness Forum,42.942077,-81.238288,Gym / Fitness Center
2,"London, Ontario, Canada",42.953765,-81.229153,Movati Athletic London North,42.990163,-81.294998,Gym / Fitness Center
3,"London, Ontario, Canada",42.953765,-81.229153,GoodLife Fitness London Southdale,42.942747,-81.229711,Gym / Fitness Center
4,"London, Ontario, Canada",42.953765,-81.229153,YMCA,43.048256,-81.257706,Gym / Fitness Center


Here we filter out the Fitness busniesses that do not fit our criteria. That includes Yoga Studios, Weight Loss Centers, and a High School which are easily done through their Venue Category. Then we filter more by inspection for those improperly clasified under 'gym' such as gymnastics centers, karate schools, a business office, and a medical health center.
We are left with a data frame contain the names of the gym venues and their geographic locations.

In [215]:
london_gyms = london_venues[london_venues["Venue Category"] != "Yoga Studio"]
london_gyms = london_gyms[london_gyms["Venue Category"] != "Weight Loss Center"]
london_gyms = london_gyms[london_gyms["Venue Category"] != "High School"]
london_gyms = london_gyms.drop([10,24,27,29,38,39,48], axis=0)
london_gyms = london_gyms.drop(["Neighbourhood", "Neighbourhood Latitude", "Neighbourhood Longitude", "Venue Category"], axis=1).reset_index(drop=True)
london_gyms.head()

,Venue,Venue Latitude,Venue Longitude
0,YMCA,42.985099,-81.241463
1,The Fitness Forum,42.942077,-81.238288
2,Movati Athletic London North,42.990163,-81.294998
3,GoodLife Fitness London Southdale,42.942747,-81.229711
4,YMCA,43.048256,-81.257706


Now we use folium to plot the gym venues over a map of London, Ontario, Canada

In [216]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers for each of the gyms to map
for lat, lng, label in zip(london_gyms['Venue Latitude'], london_gyms['Venue Longitude'], london_gyms['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

**Results & Discussion**

Our analysis shows that although there are a great number of gyms in London, Ontario, Canada (37 for a city of total population below 400K). While much of the city does seem to be saturated, there are a few areas of interest where one might find space to locate a gym. First is a significantly large open are north-west of the city center without a gym marked for significant radius. Unfortunately, Western University lies within this area and it does have a significantly large gym for student and staff use. Many people living in this area are likely students or university staff so they would likely served by the school. Potential areas in the far west and east also look like possible candidates but may not have enough through traffic in their area to drive great interest and many people from these areas may also be driving through to the city center for work.

One area that does seem central and is not currently over served is south-west of the city centre. It is central enough to have significant thru-traffic on a major route straight through to downtown and the neighbourhood residences would are already having to drive several kilometers to the nearest gym.



**Conclusion**

Despite the few possibilities that were found, it is the recommendation of the researchers that a prospective gym owner rethink London, Ontario, Canada as a location to open their gym as it appears to be currently over served. Of the possibilities, a location in the immediate south-west of the city centre may be the best choice but would not be recommeded unless the owners are prepared for significant financial investment to compete for customers.